In [1]:
import pandas as pd
import requests
import urllib.request as req
import json
from bs4 import BeautifulSoup
import requests.packages.urllib3
import json
import re

requests.packages.urllib3.disable_warnings() #使用 requests 時發生 InsecureRequestWarning: Unverified HTTPS request is being made

## **行銷策略/商品特色/商品熱門度/種類/標籤**

In [2]:
def seller(url):
    sell = []
    request = req.Request(url, headers = {
    "cookie": "adult_confirmed=1",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    soup = BeautifulSoup(data, "html.parser")

    for i in range(7):
        info = soup.select('.content')[i]
        sell.append(info.getText())
    return sell

In [3]:
def promo_feature_badges(i):
    a = []
    ad = []    
    for x in range(len(i)):                 
        a.append(i[x]['text'])

    if len(a) == 0:
        s = '無'
    else:
        s = '、'.join(a)
    return s

In [4]:
# 商品熱門度
def product_inf(url):
  request = req.Request(url, headers = {
    "cookie": "adult_confirmed=1",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
  })
  with req.urlopen(request) as response:
    data = response.read().decode("utf-8")
  soup = BeautifulSoup(data, "html.parser")
  
  popularity = []
  list1 = [] 
  final = []
  final_pop = []
  tlist = []


  for x in range(4, 10):
    try:
      global sel_1, sel_2
      sel_1 = soup.select('.m-product-list-content')[x]
      sel_2 = sel_1.find_all('li')
      keywords = '被欣賞,已賣出,人收藏'
      for q in sel_2:
        q = q.getText()
        list1.append(q)
    except IndexError:
      pass
  
    result = [1 if any([w in l and w for w in keywords.split(',')])else 0 for l in list1]
    if len(result) == 0:
      pass

  dict_fr = dict(zip(list1, result))
  for k, v in dict_fr.items():
    if v !=0:
      final.append(k)                                                                                                                                         

  try:
    for f in final:
      f = re.sub("\D", "", f)
      popularity.append(f)
    ap = popularity[0]
    sol = popularity[1]
    col = popularity[2]
    final_pop.append(ap)
    final_pop.append(sol)
    final_pop.append(col)
  except IndexError:
    for f in final:
      f = re.sub("\D", "", f)
      popularity.append(f)
    ap = popularity[0]
    col = popularity[1]
    final_pop.append(ap)
    final_pop.append(0)
    final_pop.append(col)

  condition = {'data-translate': 'description'} #將屬性名稱(例如data-開頭)與值放進一個字典中，再將此字典設定給 attrs 參數
  titles = soup.find_all(attrs = condition)
  for p in titles:
    p = p.getText().replace("\n", "")
    final_pop.append(p)

  type = soup.select('.g-breadcrumb-v2 a')
  for t in type:
    final_pop.append(t.getText())
  while "" in final_pop:
    final_pop.remove("")
  
  tag = soup.select('.m-tag')
  for i in tag:
    tlist.append(i.getText())
  final_pop.append(tlist)
  return final_pop

## **開始爬蟲**

In [5]:
import time
title = []
video = []
price = []
ad = []
ad_badge = []
pro_badge = []
fea_badge = []
browse = []
sold = []
collect = []
description = []
maincategory = []
secondary = []
minorcategory = []
material = []
tag = []
reviews = []
score = []
seller_products = []
seller_sold = []
seller_follow = []
seller_online = []
seller_response_rate = []
seller_response_speed = []
seller_ship_speed= []

for c in [0, 5]:
    print('---換種類---')
    for page in range(1, 31):
        print("正在爬第{}頁...".format(page))
        url = 'https://www.pinkoi.com/apiv3/search/search_product?page=' + str(page) + '&catp=group_' + str(c) + '&front_busting=111'
        request = req.Request(url, headers = {
            "cookie": "adult_confirmed=1",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
        })
        with req.urlopen(request) as response:
            data = response.read().decode("utf-8")
        data = json.loads(data)
        for i in range(len(data['products'])):
            tid = data['products'][i]['tid']
            sellerid = data['products'][i]['owner']
            url = 'https://www.pinkoi.com/product/' + tid
            url2 = 'https://www.pinkoi.com/store/' + sellerid
            title.append(data['products'][i]['title'])                                        #商品名
            video.append(data['products'][i]['video'])                                        #是否有展示影片
            price.append(data['products'][i]['price'])                                        #價格
            ad.append(data['products'][i]['_ad'])                                          #是否投放廣告
            ad_badge.append(data['products'][i]['_ad_badge_visible'])                            #能見廣告
                                                                        
            pro_badge.append(promo_feature_badges(data['products'][i]['promo_badges']))                   #宣傳優惠
            fea_badge.append(promo_feature_badges(data['products'][i]['feature_badges']))                 #商品特點
            
            
            product = product_inf(url)
            browse.append(product[0])
            sold.append(product[1])
            collect.append(product[2])
            description.append(product[3])
            maincategory.append(product[4])
            secondary.append(product[5])
            minorcategory.append(product[6])

            seller_inf = seller(url2)
            seller_products.append(seller_inf[0])
            seller_sold.append(seller_inf[1])
            seller_follow.append(seller_inf[2])
            seller_response_rate.append(seller_inf[4])
            seller_response_speed.append(seller_inf[5])
            seller_ship_speed.append(seller_inf[6])

            if len(product) == 9:
                material.append(product[7])
                tag.append(product[8])
            if len(product) == 8:
                material.append(None)
                tag.append(product[7])            

            try:
                reviews.append(data['products'][i]['review_info']['total'])
                score.append(data['products'][i]['review_info']['score'])
            except TypeError:
                reviews.append("沒有人評分")
                score.append(0)
        time.sleep(3)

dic = {
    "商品名稱": title,
    "商品價格": price,
    "商品瀏覽數": browse,
    "商品賣出件數": sold,
    "商品收藏人數": collect,
    "商品介紹": description,
    "商品大分類": maincategory,
    "商品中分類": secondary,
    "商品小分類": minorcategory,
    "商品材質": material,
    "商品標籤": tag,
    "展示影片": video,
    "投放廣告": ad,
    "能見廣告": ad_badge,
    "宣傳優惠": pro_badge,
    "商品特點": fea_badge,
    "評價人數": reviews,
    "評價分數": score,
    "賣家商品數量": seller_products,
    "賣家賣出件數": seller_sold,
    "賣家關注人數": seller_follow,
    "賣家回應率": seller_response_rate,
    "賣家回應速度": seller_response_speed,
    "賣家出貨速度": seller_ship_speed
}


df_product = pd.DataFrame(dic)
df_product.to_csv('C:/Users/CHARLIE/Desktop/畢專/商品資訊(新)/商品資訊(更新)_3.csv')

---換種類---
正在爬第1頁...
正在爬第2頁...
正在爬第3頁...
正在爬第4頁...
正在爬第5頁...
正在爬第6頁...
正在爬第7頁...
正在爬第8頁...
正在爬第9頁...
正在爬第10頁...
正在爬第11頁...
正在爬第12頁...
正在爬第13頁...
正在爬第14頁...
正在爬第15頁...
正在爬第16頁...
正在爬第17頁...
正在爬第18頁...
正在爬第19頁...
正在爬第20頁...
正在爬第21頁...
正在爬第22頁...
正在爬第23頁...
正在爬第24頁...
正在爬第25頁...
正在爬第26頁...
正在爬第27頁...
正在爬第28頁...
正在爬第29頁...
正在爬第30頁...
---換種類---
正在爬第1頁...
正在爬第2頁...
正在爬第3頁...
正在爬第4頁...
正在爬第5頁...
正在爬第6頁...
正在爬第7頁...
正在爬第8頁...
正在爬第9頁...
正在爬第10頁...
正在爬第11頁...
正在爬第12頁...
正在爬第13頁...
正在爬第14頁...
正在爬第15頁...
正在爬第16頁...
正在爬第17頁...
正在爬第18頁...
正在爬第19頁...
正在爬第20頁...
正在爬第21頁...
正在爬第22頁...
正在爬第23頁...
正在爬第24頁...
正在爬第25頁...
正在爬第26頁...
正在爬第27頁...
正在爬第28頁...
正在爬第29頁...
正在爬第30頁...


In [6]:
df_product

,商品名稱,商品價格,商品瀏覽數,商品賣出件數,商品收藏人數,商品介紹,商品大分類,商品中分類,商品小分類,商品材質,...,宣傳優惠,商品特點,評價人數,評價分數,賣家商品數量,賣家賣出件數,賣家關注人數,賣家回應率,賣家回應速度,賣家出貨速度
0,【一套兩件】客製化情侶裝香港原創短袖T恤婚攝 - 食住你,$$TWD$$1455$$,28748,43,516,★ 品牌故事氹人，是必需的。giveco氹人專門店，以氹人開心為己任。無論人見人愛嘅禮品賀卡...,衣著良品,女上衣,女素色/印花 T 恤,棉．麻,...,無,可客製,12,5.0,94,161,226,96%,1 日內,7 日以上
1,【可在家洗和日本製造】象牙色 優雅的斗篷襯衫,$$TWD$$3388$$,955,0,15,[產品名稱] Elegant Cape Blouse / 優雅的斗篷襯衫[顏色] IVORY...,衣著良品,女上衣,女襯衫,環保材質,...,免運、88 折,無,沒有人評分,0.0,55,16,60,100%,1 日內,1～3 日
2,TopBasics 日系復古水洗垂感衛衣,$$TWD$$1267$$,1138,12,18,簡介:• 單品在面料上做了水洗復古效果，更能體驗出一種帥痞的造型風格。• 選用100%棉面料...,衣著良品,中性服飾,帽T/衛衣,棉．麻,...,免運、88 折,無,沒有人評分,0.0,130,517,"1,190",96%,1 日內,一週內
3,【客製化禮物】情人客製 頭像睡衣 心心款 女朋友禮物,$$TWD$$1539$$,18079,155,327,https://live.staticflickr.com/65535/5213585490...,衣著良品,居家服飾/內衣褲,家居服/睡衣,其他人造纖維,...,免運,可客製、15 人正準備購買,12,4.9,37,322,171,100%,1 日內,7 日以上
4,棉毛混紡彩色格紋細壓褶開襟落肩小立領百褶袖口長版襯衫洋裝,$$TWD$$4580$$,967,1,17,🔺タックタック tuck tuck🔺タック tuck 摺疊布料車出褶縫線的意思做了好幾款立體...,衣著良品,女下身,洋裝/連身裙,棉．麻,...,免運,無,沒有人評分,0.0,61,224,605,100%,1 日內,7 日以上
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,自然家 | 原木色竹時日式掛鐘 壁掛靜音時鐘 木質鐘掛鐘,$$TWD$$1655$$,22964,36,670,**商品尺寸較大，只提供宅配服務**https://img.alicdn.com/imgex...,居家生活,家用電器/用品,時鐘/鬧鐘,木頭,...,免運、88 折,無,13,5.0,23,"26,473","5,897",90%,數小時內,7 日以上
4796,Rosemary小小花園玻璃罩-鉑金爵士 永生花/情人節/生日/畢業禮物,$$TWD$$380$$,412,2,13,**材料說明：約80%永生花。20%乾燥花。混合設計，玻璃罩為軟木塞款****尺寸：∅6.5...,居家生活,家飾佈置,乾燥花/永生花,植物．花,...,無,獨家販售,2,5.0,15,86,53,100%,1 日內,1～3 日
4797,水泥摩艾 Moai－嘘嘴摩艾 摩艾筆筒 多肉植物盆器,$$TWD$$585$$,5079,28,144,◖商品說明◗★售價是一個的價格，均不含拍攝配件。粗獷摩艾巨石像，先以手工捏塑成型，在以水泥材...,居家生活,家飾佈置,玩偶/公仔,水泥,...,無,無,5,5.0,53,"2,006","1,782",97%,1 日內,1 日內
4798,靜蓮│生日祝壽禮物、新居入厝、佛教界交流伴手禮 │素雅系列,$$TWD$$4488$$,1525,2,16,**可運送境外，須補運費**網站所在國為台灣，所有計價配送費用限臺灣本島。臺灣外島、 中國大...,居家生活,家飾佈置,海報/掛畫/掛布,24k 金,...,免運、88 折,可客製,沒有人評分,0.0,299,339,448,98%,1 日內,1～3 日
